In [41]:
##metaheuristique
import numpy as np
import requests
import random

In [5]:
##TELECHARGEMENT DATA

base_url = "https://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/mknapcb"


for i in range(9):
    url = base_url + str(i+1) + ".txt"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Vérifie qu'il n'y a pas d'erreur HTTP
        contenu_texte = response.text

        with open("mknapcb" + str(i+1) + ".txt", "w") as fichier:
            fichier.write(contenu_texte)


    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de l'ouverture de l'URL : {e}")

In [6]:
## EXTRACTION DATA

def extract_data(mon_fichier):
    with open(mon_fichier, "r") as fichier:
        contenu = fichier.read()
        contenu = contenu.split()
        #print(contenu)
        nb_instances = int(contenu[0])
        #print(nb_instances)
        instances = {}
        compteur = 1
        for i in range(1,nb_instances+1):
            elements = contenu[compteur:]
            n  = int(elements[0])
            #print(n)
            m = int(elements[1])
            #print(m)
            val_opt = float(elements[2])
            cost = np.array([float(elements[e]) for e in range(3,3+n)])
            #print(cost)
            a = np.array([int(elements[e]) for e in range(3+n,3+n+n*m)])
            a = a.reshape(m,n)
            b = np.array([int(elements[e]) for e in range(3+n+n*m,3+n+n*m+m)])
            instances[i] = {}
            instances[i]["Nb projet"] = n
            instances[i]["Nb ressource"] = m
            instances[i]["Valeur optimale"] = val_opt
            instances[i]["cost"] = cost
            instances[i]["A"] = a
            instances[i]["B"] = b
            compteur += 3+n+m*n+m
        return nb_instances, instances

In [194]:
## VISUALISATION DATA

mon_fichier = "C:/Users/Thibaud/3A/metaheuristique/Data/mknap1.txt"
nb_instances, instances = extract_data(mon_fichier)

print(nb_instances)
print(instances)

7
{1: {'Nb projet': 6, 'Nb ressource': 10, 'Valeur optimale': 3800.0, 'cost': array([ 100.,  600., 1200., 2400.,  500., 2000.]), 'A': array([[ 8, 12, 13, 64, 22, 41],
       [ 8, 12, 13, 75, 22, 41],
       [ 3,  6,  4, 18,  6,  4],
       [ 5, 10,  8, 32,  6, 12],
       [ 5, 13,  8, 42,  6, 20],
       [ 5, 13,  8, 48,  6, 20],
       [ 0,  0,  0,  0,  8,  0],
       [ 3,  0,  4,  0,  8,  0],
       [ 3,  2,  4,  0,  8,  4],
       [ 3,  2,  4,  8,  8,  4]]), 'B': array([80, 96, 20, 36, 44, 48, 10, 18, 22, 24])}, 2: {'Nb projet': 10, 'Nb ressource': 10, 'Valeur optimale': 8706.1, 'cost': array([ 600.1,  310.5, 1800. , 3850. ,   18.6,  198.7,  882. , 4200. ,
        402.5,  327. ]), 'A': array([[ 20,   5, 100, 200,   2,   4,  60, 150,  80,  40],
       [ 20,   7, 130, 280,   2,   8, 110, 210, 100,  40],
       [ 60,   3,  50, 100,   4,   2,  20,  40,   6,  12],
       [ 60,   8,  70, 200,   4,   6,  40,  70,  16,  20],
       [ 60,  13,  70, 250,   4,  10,  60,  90,  20,  24],
       

In [52]:
## HEURISTIQUE

def heuristique_sac_a_dos(n, m, cost, a, b, fct_voisinage):
    x= np.zeros(n)
    #print('xtype', type(x)) 
    b_prime = np.sum(b) #somme des ressources
    #print('b_prime=', b_prime) 
    a_prime= np.sum(a, axis=0) #somme des ressources nécessaires pour chaque projet
    #print('a_prime=', a_prime)
    y = -cost/a_prime #signe - pour trier dans l'ordre décroissant
    #print(y)
    indices =y.argsort()
    #print('indices', indices)
    ressource = 0
    index = []
    for i in indices:
        if ressource + a_prime[i] <= b_prime:
            x[i] = 1
            ressource += a_prime[i]
            #print('i=', i)
            #print('ressource', ressource)
            #print('b_prime', b_prime)
            index.append(i) 
    #print("x before dot (a,x) ", x)
    #verification solution réalisable + réparation
    
    while any(np.dot(a, x) > b):
        #print('x=', x)
        #print('p=',(np.dot(a, x)))
        #print('b=',b)
        last_index = index[-1]
        x[last_index] = 0
        #print(x)
        index.pop()
    #print("x after dot (a,x) ", x)
    value = np.dot(cost, x)

    return x, value

In [53]:
##TEST HEURISTIQUE

mon_fichier = "C:/Users/Thibaud/3A/metaheuristique/Data/mknap1.txt"
nb_instances, instances = extract_data(mon_fichier)

for i in range(1,nb_instances+1):
    a = instances[i]["A"]
    b = instances[i]["B"]
    cost = instances[i]["cost"]
    n = instances[i]["Nb projet"]
    m = instances[i]["Nb ressource"]
    val_opt = instances[i]["Valeur optimale"]
    x, value =heuristique_sac_a_dos(n, m, cost, a, b,1)
    print('val_opt=', val_opt, 'value=', value, x)

val_opt= 3800.0 value= 3800.0 [0. 1. 1. 0. 0. 1.]
val_opt= 8706.1 value= 8318.3 [1. 1. 1. 0. 0. 1. 1. 1. 0. 1.]
val_opt= 4015.0 value= 3825.0 [1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1.]
val_opt= 6120.0 value= 5760.0 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.]
val_opt= 12400.0 value= 12150.0 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1.]
val_opt= 10618.0 value= 8771.0 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
val_opt= 16537.0 value= 15614.0 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1.
 1. 1.]


In [212]:
chemin = "C:/Users/Thibaud/3A/metaheuristique/Data/result.txt"

with open(chemin, "w") as fichier:
    fichier.write("RESULT\n\n")

In [237]:
## TELECHARGEMENT SOLUTIONS HEURISTIQUE GLOUTONNE


mon_fichier = "C:/Users/Thibaud/3A/metaheuristique/Data/mknap1.txt"
nb_instances, instances = extract_data(mon_fichier)
with open(chemin, "a", encoding="utf-8") as f:
    f.write(f"FICHIER : mknap1 \n\n")
for j in range(1,nb_instances+1):
    a = instances[j]["A"]
    b = instances[j]["B"]
    cost = instances[j]["cost"]
    n = instances[j]["Nb projet"]
    m = instances[j]["Nb ressource"]
    val_opt = instances[j]["Valeur optimale"]
    x, value =heuristique_sac_a_dos(n, m, cost, a, b, fct_voisinage=1)
    chemin = "C:/Users/Thibaud/3A/metaheuristique/Data/result.txt"
    with open(chemin, "a", encoding="utf-8") as f:
        f.write(f"instance {j} : {value} \n")
        f.write(f"optimal value : {val_opt} \n")
        f.write(f"gap : {(val_opt-value)/val_opt} \n")
        f.write(f"projet selectionné : {x} \n")
        f.write("\n")
    

In [238]:
## TELECHARGEMENT SOLUTIONS HEURISTIQUE GLOUTONNE

base_fichier = "C:/Users/Thibaud/3A/metaheuristique/Data/mknapcb"

for i in range(9):
    mon_fichier = base_fichier + str(i+1) + ".txt"
    nb_instances, instances = extract_data(mon_fichier)
    with open(chemin, "a", encoding="utf-8") as f:
        f.write(f"FICHIER : mknapcb{i+1} \n\n")
    for j in range(1,nb_instances+1):
        a = instances[j]["A"]
        b = instances[j]["B"]
        cost = instances[j]["cost"]
        n = instances[j]["Nb projet"]
        m = instances[j]["Nb ressource"]
        val_opt = instances[j]["Valeur optimale"]
        x, value =heuristique_sac_a_dos(n, m, cost, a, b, fct_voisinage=1)
        chemin = "C:/Users/Thibaud/3A/metaheuristique/Data/result.txt"
        with open(chemin, "a", encoding="utf-8") as f:
            f.write(f"instance {j} : {value} \n")
            f.write(f"optimal value : {val_opt} \n")
            f.write(f"gap : {(val_opt-value)/val_opt} \n")
            f.write(f"projet selectionné : {x} \n")
            f.write("\n")


C:\Users\Thibaud\AppData\Local\Temp\ipykernel_6740\743228389.py:22: RuntimeWarning: divide by zero encountered in scalar divide
  f.write(f"gap : {(val_opt-value)/val_opt} \n")


In [59]:
## DEFINITION D'UN VOISINAGE

def is_realisable(x, a, b):
    return all(np.dot(a, x) <= b)

def voisinage(x):
    voisins = []
    for i in range(len(x)):
        if x[i] == 0:
            x_voisin = x.copy()
            x_voisin[i] = 1
            voisins.append(x_voisin)
        else:
            x_voisin = x.copy()
            x_voisin[i] = 0
            voisins.append(x_voisin)
    return voisins

def voisinage_echange(x):
    voisins = []
    for i in range(len(x)):
        if x[i] == 1:
            for j in range(len(x)):
                if x[j] == 0:
                    x_voisin = x.copy()
                    x_voisin[i] = 0
                    x_voisin[j] = 1
                    voisins.append(x_voisin)
    return voisins

def voisin_realisable(x, a, b, fct_voisinage):
    voisins = fct_voisinage(x)
    voisins_realisables = []
    for voisin in voisins:
        if is_realisable(voisin, a, b):
            voisins_realisables.append(voisin)
    return voisins_realisables

In [58]:
## ALGORITHME DE MONTEE


def algorithme_montee(xmax, a, b, cost, fct_voisinage):
    fin = False
    while not fin:
        voisins = voisin_realisable(xmax, a, b, fct_voisinage)
        fin = True
        value = np.dot(cost, xmax)
        for voisin in voisins:
            value_voisin = np.dot(cost, voisin)
            if value_voisin > value:
                xmax = voisin
                value = value_voisin
                fin = False
    return xmax, value


In [54]:
##TEST ALGORITHME DE MONTEE

mon_fichier = "C:/Users/Thibaud/3A/metaheuristique/Data/mknap1.txt"
nb_instances, instances = extract_data(mon_fichier)
num_instance = 5
a = instances[num_instance]["A"]
b = instances[num_instance]["B"]
cost = instances[num_instance]["cost"]
n = instances[num_instance]["Nb projet"]
m = instances[num_instance]["Nb ressource"]
val_opt = instances[num_instance]["Valeur optimale"]
x, value =heuristique_sac_a_dos(n, m, cost, a, b, fct_voisinage=1)
xmax, value_monte = algorithme_montee(x, a, b, cost, voisinage)
print('val_opt=', val_opt, 'value=', value, 'value_monte=', value_monte)
print('x_type', type(x))
print('xmax_type', type(xmax))


val_opt= 12400.0 value= 12150.0 value_monte= 12400.0
x_type <class 'numpy.ndarray'>
xmax_type <class 'numpy.ndarray'>


In [12]:
## AFFICHAGE SOLUTION ALGORITHME DE MONTEE

mon_fichier = "C:/Users/Thibaud/3A/metaheuristique/Data/mknap1.txt"
nb_instances, instances = extract_data(mon_fichier)

L_value=[]
L_value_monte=[]
L_value_opt=[]

for i in range(1,nb_instances+1):
    a = instances[i]["A"]
    b = instances[i]["B"]
    cost = instances[i]["cost"]
    n = instances[i]["Nb projet"]
    m = instances[i]["Nb ressource"]
    val_opt = instances[i]["Valeur optimale"]
    x, value =heuristique_sac_a_dos(n, m, cost, a, b, fct_voisinage=1)
    xmax, value_monte = algorithme_montee(x, a, b, cost,voisinage)
    L_value.append(value)
    L_value_monte.append(value_monte)
    L_value_opt.append(val_opt)
    print('val_opt=', val_opt,  'value_monte=', value_monte,'value_heuristique=', value)



val_opt= 3800.0 value_monte= 3800.0 value_heuristique= 3800.0
val_opt= 8706.1 value_monte= 8336.9 value_heuristique= 8318.3
val_opt= 4015.0 value_monte= 3825.0 value_heuristique= 3825.0
val_opt= 6120.0 value_monte= 5965.0 value_heuristique= 5760.0
val_opt= 12400.0 value_monte= 12400.0 value_heuristique= 12150.0
val_opt= 10618.0 value_monte= 10209.0 value_heuristique= 8771.0
val_opt= 16537.0 value_monte= 15832.0 value_heuristique= 15614.0


In [57]:
## DEUXIEME VOISINAGE : échange de projet

mon_fichier = "C:/Users/Thibaud/3A/metaheuristique/Data/mknapcb1.txt"
nb_instances, instances = extract_data(mon_fichier)

L_value=[]
L_value_monte=[]
L_value_opt=[]

for i in range(1,nb_instances+1):
    a = instances[i]["A"]
    b = instances[i]["B"]
    cost = instances[i]["cost"]
    n = instances[i]["Nb projet"]
    m = instances[i]["Nb ressource"]
    val_opt = instances[i]["Valeur optimale"]
    x, value =heuristique_sac_a_dos(n, m, cost, a, b, fct_voisinage=1)
    xmax, value_monte = algorithme_montee(x, a, b, cost, voisinage_echange)
    L_value.append(value)
    L_value_monte.append(value_monte)
    L_value_opt.append(val_opt)
    print('val_opt=', val_opt,  'value_monte=', value_monte,'value_heuristique=', value)



val_opt= 0.0 value_monte= 22645.0 value_heuristique= 20327.0
val_opt= 0.0 value_monte= 24030.0 value_heuristique= 23159.0
val_opt= 0.0 value_monte= 22835.0 value_heuristique= 21156.0
val_opt= 0.0 value_monte= 21382.0 value_heuristique= 17379.0
val_opt= 0.0 value_monte= 23071.0 value_heuristique= 20932.0
val_opt= 0.0 value_monte= 24523.0 value_heuristique= 24042.0
val_opt= 0.0 value_monte= 24814.0 value_heuristique= 22827.0
val_opt= 0.0 value_monte= 22958.0 value_heuristique= 21949.0
val_opt= 0.0 value_monte= 23838.0 value_heuristique= 20744.0
val_opt= 0.0 value_monte= 23416.0 value_heuristique= 21472.0
val_opt= 0.0 value_monte= 42420.0 value_heuristique= 41091.0
val_opt= 0.0 value_monte= 41256.0 value_heuristique= 38894.0
val_opt= 0.0 value_monte= 41717.0 value_heuristique= 40385.0
val_opt= 0.0 value_monte= 44586.0 value_heuristique= 41991.0
val_opt= 0.0 value_monte= 41235.0 value_heuristique= 39855.0
val_opt= 0.0 value_monte= 42535.0 value_heuristique= 42030.0
val_opt= 0.0 value_monte

In [99]:
## ALOGORITHE GENETIQUE

def perturber_solution(x):
    n = x.size
    for _ in range(np.random.randint(1, 2+ n//10)):
        index = np.random.randint(len(x))
        x[index] = 1 - x[index]
    return x

def fit(x, a, b, cost):
    if not is_realisable(x, a, b):
        return -1
    return np.dot(cost, x)

def reparation_v2(x, a, b, cost):
    b_prime = np.sum(b) 
    a_prime= np.sum(a, axis=0) 
    y = -cost/a_prime 
    indices =y.argsort()
    index_to_pop = [i for i in indices if x[i] == 1]
    while any(np.dot(a, x) > b):
        last_index = index_to_pop[-1]
        x[last_index] = 0
        #print(x)
        index_to_pop.pop()
    return x

def generation_pop(n, m, cost, a, b, taille_pop, generation_solution, fct_voisinage):   
    x = generation_solution(n, m, cost, a, b, fct_voisinage)[0]
    x = algorithme_montee(x, a, b, cost, fct_voisinage)[0]
    popu = [x]
    for _ in range (taille_pop-1):
        x = perturber_solution(x)
        popu.append(x)
    return popu

def algorithme_genetique(n, m, cost, a, b, nb_iter, taille_pop, max_pop, taux_mut, generation_solution, fct_voisinage):
    max_pop = (max_pop//2)*2
    #population initiale
    pop = generation_pop(n, m, cost, a, b, taille_pop, generation_solution, fct_voisinage)
    random.shuffle(pop)
    #iterations
    for _ in range(nb_iter):
        fit_value = [fit(x, a, b, cost) for x in pop]
        proba = [fit_value[i]/sum(fit_value) for i in range(len(fit_value))]
        parents = []
        #selection parents
        while len(parents) < max_pop:
            for i in range(len(pop)):
                if np.random.rand() < proba[i]:
                    parents.append(pop[i])
        parents = parents[:max_pop]
        #croisement
        #1-point crossover
        child = []
        for i in range(0, len(parents), 2):
            cut = np.random.randint(1, n)
            #print(len(parents[i]))
            #print(len(parents[i+1]))
            child1 = np.concatenate((parents[i][:cut], parents[i+1][cut:]))
            child2 = np.concatenate((parents[i+1][:cut], parents[i][cut:]))
            #print('x',child2.shape)
            child.append(child1)
            child.append(child2)
        #2-point crossover
        random.shuffle(parents)
        for i in range(0, len(parents), 2):
            p1, p2 = sorted(np.random.choice(range(n), size=2, replace=False))
            child1 = np.concatenate((parents[i][:p1], parents[i+1][p1:p2], parents[i][p2:]))
            child2 = np.concatenate((parents[i+1][:p1], parents[i][p1:p2], parents[i+1][p2:]))
            child.append(child1)
            child.append(child2)
            #print('x',child2.shape)
        for j in range(len(child)):
            if not is_realisable(child[j], a, b):
                child[j] = reparation_v2(child[j], a, b, cost)
        #mutation
        for j in range(len(child)):
            if np.random.rand() < taux_mut:
                child[j] = perturber_solution(child[j])
        pop = parents + child
        random.shuffle(pop)
    valeurs = [fit(x, a, b, cost) for x in pop]
    index = valeurs.index(max(valeurs))
    x= pop[index]
    value = np.dot(cost, pop[index])
    return x, value



In [101]:
##TEST ALGORITHME GENETIQUE

mon_fichier = "C:/Users/Thibaud/3A/metaheuristique/Data/mknapcb1.txt"

nb_instances, instances = extract_data(mon_fichier)
num_instance = 4
a = instances[num_instance]["A"]
b = instances[num_instance]["B"]
cost = instances[num_instance]["cost"]
n = instances[num_instance]["Nb projet"]
m = instances[num_instance]["Nb ressource"]
val_opt = instances[num_instance]["Valeur optimale"]
#x, value =heuristique_sac_a_dos(n, m, cost, a, b, fct_voisinage=1)
#xmax, value_monte = algorithme_montee(x, a, b, cost, voisinage)
xgen, value_gen = algorithme_genetique(n, m, cost, a, b, 180, 200, 150, 0.1, heuristique_sac_a_dos, voisinage)
print('val_opt=', val_opt, 'value=', value, 'value_monte=', value_monte, 'value_gen=', value_gen)

val_opt= 0.0 value= 17379.0 value_monte= 20939.0 value_gen= 20608.0
